<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/gaming_behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

線上遊戲行為資料

https://www.kaggle.com/datasets/rabieelkharoua/predict-online-gaming-behavior-dataset

## KMeans

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14X-91TfbqcSULUoBBraqYLKW_anoBlBmZfOssYd620c/edit?gid=499173031#gid=499173031')

In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('online_gaming_behavior_dataset').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_expanded = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_expanded.head()

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.27111876,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961381,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755243,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351277,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.53194452,0,Medium,2,131,95,37,Medium


In [ ]:
type(df_expanded['PlayTimeHours'][0]) #確認時間格式

str

In [ ]:

# 如果有 PlayerID 和 Gender 欄位就保留，否則就略過
# 這裡不用轉換 Gender，但可以檢查是否存在
if 'PlayerID' in df_expanded.columns:
    df_expanded['PlayerID'] = df_expanded['PlayerID']

if 'Gender' in df_expanded.columns:
    df_expanded['Gender'] = df_expanded['Gender']

# 去除關鍵欄位的缺失值
df_expanded = df_expanded.dropna(subset=['Age', 'PlayTimeHours', 'InGamePurchases'])

# 顯示前五筆資料
df_expanded.head()


,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
0,9000,43,Male,Other,Strategy,16.27111876,0,Medium,6,108,79,25,Medium
1,9001,29,Female,USA,Strategy,5.525961381,0,Medium,5,144,11,10,Medium
2,9002,22,Female,USA,Sports,8.223755243,0,Easy,16,142,35,41,High
3,9003,35,Male,USA,Action,5.265351277,1,Easy,9,85,57,47,Medium
4,9004,33,Male,Europe,Action,15.53194452,0,Medium,2,131,95,37,Medium


In [ ]:
# 提取所需的欄位
player_activity_info = df_expanded[['PlayerID', 'Age', 'Gender', 'PlayTimeHours', 'InGamePurchases']]

# 檢查是否需要進一步處理或轉換數據
# 比如如果 'InGamePurchases' 需要轉換為數字，確保數據格式正確
player_activity_info['InGamePurchases'] = pd.to_numeric(player_activity_info['InGamePurchases'], errors='coerce')

# 如果需要處理缺失值，這裡可以進一步過濾或填補缺失數據
player_activity_info = player_activity_info.dropna()


<ipython-input-8-7a1e34b50dbb>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_activity_info['InGamePurchases'] = pd.to_numeric(player_activity_info['InGamePurchases'], errors='coerce')


In [ ]:
player_activity_info

,PlayerID,Age,Gender,PlayTimeHours,InGamePurchases
0,9000,43,Male,16.27111876,0
1,9001,29,Female,5.525961381,0
2,9002,22,Female,8.223755243,0
3,9003,35,Male,5.265351277,1
4,9004,33,Male,15.53194452,0
...,...,...,...,...,...
96,9096,28,Male,9.208410715,1
97,9097,17,Male,8.644038309,0
98,9098,15,Female,5.970601392,0
99,9099,19,Female,15.37717222,0


STR轉數字，One-Hot Encoding指把 Gender（性別）這個文字欄位轉換成機器可以懂的「0/1」數字欄位。其餘數字資料欄位轉資料格式

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 假設你已經有 player_activity_info 這個 DataFrame

# Step 1: 做 One-Hot Encoding（性別欄位）
df_encoded = pd.get_dummies(player_activity_info, columns=['Gender'])
# 指定要轉換的欄位
numeric_columns = ['Age', 'PlayTimeHours', 'InGamePurchases']

# 使用 pd.to_numeric 強制轉換（錯的值設為 NaN）
df_encoded[numeric_columns] = df_encoded[numeric_columns].apply(pd.to_numeric, errors='coerce')


In [ ]:
df_encoded

,PlayerID,Age,PlayTimeHours,InGamePurchases,Gender_Female,Gender_Male
0,9000,43,16.271119,0,False,True
1,9001,29,5.525961,0,True,False
2,9002,22,8.223755,0,True,False
3,9003,35,5.265351,1,False,True
4,9004,33,15.531945,0,False,True
...,...,...,...,...,...,...
96,9096,28,9.208411,1,False,True
97,9097,17,8.644038,0,False,True
98,9098,15,5.970601,0,True,False
99,9099,19,15.377172,0,True,False


標準化資料(不知道大家有沒有學過標準差了...)

把每個欄位轉換成「平均值為 0，標準差為 1」的標準格式。因為今天數據可能是1，也可能是50，如果不先標準化，KMeans 分群會被數字比較大的欄位影響。

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 要標準化的欄位
features = ['Age', 'PlayTimeHours', 'InGamePurchases', 'Gender_Female', 'Gender_Male']

# 標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_encoded[features])

# 執行 KMeans 分群（這裡舉例分 4 群）
kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')
df_encoded['Cluster'] = kmeans.fit_predict(scaled_features)


In [ ]:
import plotly.express as px

# 計算各群人數
cluster_counts = df_encoded['Cluster'].value_counts().reset_index()
cluster_counts.columns = ['Cluster', 'Count']

# 畫長條圖
fig = px.bar(cluster_counts, x='Cluster', y='Count',
             title="各群組人數分佈（KMeans）",
             labels={'Cluster': '群組', 'Count': '人數'},
             color='Cluster')

# 美化版面
fig.update_layout(xaxis_title="群組", yaxis_title="人數", title_x=0.5)
fig.show()


In [ ]:
#透過程式碼我們可以查看不同分群的資料
group = df_encoded[df_encoded['Cluster'] == 2]
group

,PlayerID,Age,PlayTimeHours,InGamePurchases,Gender_Female,Gender_Male,Cluster
8,9008,38,18.152733,0,True,False,2
9,9009,38,23.942772,0,True,False,2
18,9018,26,19.416079,0,True,False,2
19,9019,36,19.338826,0,True,False,2
21,9021,41,16.167690,0,True,False,2
30,9030,18,22.858673,0,True,False,2
32,9032,28,14.527639,0,True,False,2
35,9035,16,17.491258,0,True,False,2
59,9059,30,15.154139,0,True,False,2
61,9061,38,19.463564,0,True,False,2



由資料得知：

1. g0是課金的女玩家(8)
2. g1是男玩家(64)
3. g2是沒課金的女玩家(15)
4. g3是沒課金的女玩家(14)


問題：g2、3的區別


In [ ]:
# 看群組 2 的數值欄位平均
group2 = df_encoded[df_encoded['Cluster'] == 2]
group_mean = group2[['Age', 'PlayTimeHours', 'InGamePurchases']].mean()
print(group_mean)

Age                29.400000
PlayTimeHours      18.381714
InGamePurchases     0.000000
dtype: float64


In [ ]:
# 看群組 3 的數值欄位平均
group3 = df_encoded[df_encoded['Cluster'] == 3]
group_mean = group3[['Age', 'PlayTimeHours', 'InGamePurchases']].mean()
print(group_mean)

Age                27.571429
PlayTimeHours       5.433174
InGamePurchases     0.000000
dtype: float64


得知結果：g3的遊玩時間遠小於g2

## PCA

稍微擴大資料範圍，把之前的資訊都納入考量看機器會怎麼分組

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px

# ✅ Step 1: 資料預處理
df_encoded = df_expanded.copy()

# One-Hot Encoding 類別欄位
categorical_columns = ['Gender', 'Location', 'GameGenre', 'GameDifficulty', 'EngagementLevel']
df_encoded = pd.get_dummies(df_encoded, columns=categorical_columns)

# 數值欄位清單
numeric_columns = [
    'Age', 'PlayTimeHours', 'InGamePurchases',
    'SessionsPerWeek', 'AvgSessionDurationMinutes',
    'PlayerLevel', 'AchievementsUnlocked'
]

# 數值欄位強制轉成 numeric
df_encoded[numeric_columns] = df_encoded[numeric_columns].apply(pd.to_numeric, errors='coerce')

# 補上 NaN（使用中位數填補）
df_encoded[numeric_columns] = df_encoded[numeric_columns].fillna(df_encoded[numeric_columns].median())

# 去除 PlayerID（不參與分群）
features = df_encoded.drop(columns=['PlayerID'])

# 標準化
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# ✅ Step 2: PCA 降成 2 維
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_features)

df_encoded['PCA1'] = pca_result[:, 0]
df_encoded['PCA2'] = pca_result[:, 1]

# ✅ Step 3: KMeans 分群
kmeans = KMeans(n_clusters=3, random_state=0, n_init='auto')
df_encoded['Cluster'] = kmeans.fit_predict(pca_result)

# ✅ Step 4: 視覺化（PCA + KMeans）
fig = px.scatter(df_encoded, x='PCA1', y='PCA2', color='Cluster',
                 title="KMeans 分群視覺化 (使用 PCA 降維)",
                 labels={'Cluster': '群組'},
                 hover_data=['Age', 'PlayTimeHours', 'PlayerLevel'])
fig.update_layout(title_x=0.5)
fig.show()


有點難看出來這些群組的區別在哪?可以用下面的方式來分析

In [ ]:
import pandas as pd
import plotly.express as px

# Step 1: 加上 Cluster 標籤
df_with_cluster = df_encoded.copy()
df_with_cluster['Cluster'] = kmeans.labels_

# Step 2: 分群後的平均
cluster_summary = df_with_cluster.groupby('Cluster').mean(numeric_only=True)

# Step 3: 轉置 + melt 成長格式
cluster_summary_reset = cluster_summary.T.reset_index().melt(id_vars='index', var_name='Cluster', value_name='Average')
cluster_summary_reset = cluster_summary_reset.rename(columns={'index': 'Feature'})

# Step 4: 畫圖
fig = px.bar(cluster_summary_reset, x='Feature', y='Average', color='Cluster',
             title="各群組在不同特徵的平均值比較", barmode='group')
fig.update_layout(xaxis_title="特徵", yaxis_title="平均值", title_x=0.5)
fig.show()


看起來g1是遊戲公會開會的時間跟頻率比較高的組別，
g2的年齡比較高，感覺起來g2的人比較不喜歡開會，但等級比較高。g0的女生比較多(76%)

也可以用下面的方式問chatgpt我們的權重佔多少

In [ ]:
# 印出每個主成分的權重（對原始欄位的貢獻）
loadings = pd.DataFrame(
    pca.components_.T,  # 每欄是一個主成分
    columns=['PCA1', 'PCA2'],
    index=features.columns  # 原始欄位名
)
print(loadings)


                               PCA1      PCA2
Age                        0.044680 -0.202984
PlayTimeHours              0.074418  0.054814
InGamePurchases           -0.086699 -0.149926
SessionsPerWeek           -0.025558  0.428000
AvgSessionDurationMinutes -0.001426  0.368578
PlayerLevel               -0.048319 -0.135309
AchievementsUnlocked      -0.138552  0.053059
Gender_Female             -0.471509  0.074962
Gender_Male                0.471509 -0.074962
Location_Asia             -0.025616 -0.198258
Location_Europe           -0.158247 -0.074890
Location_Other             0.160024  0.036185
Location_USA               0.072037  0.181471
GameGenre_Action          -0.096346 -0.023588
GameGenre_RPG              0.097139  0.028361
GameGenre_Simulation      -0.160879 -0.110164
GameGenre_Sports          -0.099401  0.095279
GameGenre_Strategy         0.254871 -0.002954
GameDifficulty_Easy       -0.425776 -0.044818
GameDifficulty_Hard        0.116688 -0.028374
GameDifficulty_Medium      0.36729

🔍 總覽：PCA1 和 PCA2 分別代表什麼？

主成分	簡要說明


* PCA1	主要分辨「性別」與「遊戲難度」的差異。

  正值偏「男性 + Medium難度」，負值偏「女性 + Easy難度」
* PCA2	主要分辨「參與程度」與「遊戲頻率」的差異。
  
  正值偏「高參與 + 常玩」、負值偏「低參與 + 不常玩」